In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import shap
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

## EDA

In [ ]:
# fetch dataset 
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes) 
df = cdc_diabetes_health_indicators.data.features
df["Diabetes"] = cdc_diabetes_health_indicators.data.targets
X = df.drop(columns='Diabetes')
y = cdc_diabetes_health_indicators.data.targets

# Create df with features and target for graphing purposes later


# Jitter y values for graphing purposes later
y_jitter = []
for i in range(len(y)):
    y_jitter.append(y.loc[i][0] + np.random.uniform(-0.2,0.2))

# feature engineering
genhlth_dict = {1: 5, 2: 4, 3: 3, 4: 2, 5: 1}
X['GenHlth'] = X['GenHlth'].map(genhlth_dict)
age_dict = {1: 21, 2: 27, 3: 32, 4: 37, 5: 42, 6: 47, 7: 52, 8: 57, 9: 62, 10: 67, 11: 72, 12: 77, 13: 80}
X['Age'] = X['Age'].map(age_dict)
income_dict = {1: 10, 2: 12.5, 3: 17.5, 4: 22.5, 5: 30, 6: 42.5, 7: 62.5, 8: 75}
X['Income'] = X['Income'].map(income_dict)
X['MentHlth'] = 30 - X['MentHlth']
X['PhysHlth'] = 30 - X['PhysHlth']

In [ ]:
# Check for NA
print(X.isna().sum())
print(y.isna().sum())

In [ ]:
# Five Number Summaries of numeric variables
print("BMI Summary: ", np.percentile(X["BMI"], [0, 25, 50, 75, 100]))
print("MentHlth Summary: ", np.percentile(X["MentHlth"], [0, 25, 50, 75, 100]))
print("PhysHlth Summary: ", np.percentile(X["PhysHlth"], [0, 25, 50, 75, 100]))
print("Age Summary: ", np.percentile(X["Age"], [0, 25, 50, 75, 100]))
print("Education Summary: ", np.percentile(X["Education"], [0, 25, 50, 75, 100]))
print("Income Summary: ", np.percentile(X["Income"], [0, 25, 50, 75, 100]))

In [ ]:
bmi = []
for i in range(len(X["BMI"])):
    bmi.append(X["BMI"][i] + np.random.uniform(-0.2,0.2))
# Diabetes vs. BMI
plt.figure(figsize=(11,6))
plt.scatter(bmi, y_jitter, s=0.1)

In [ ]:
ment = []
for i in range(len(X["MentHlth"])):
    ment.append(X["MentHlth"][i] + np.random.uniform(-0.2,0.2))
plt.figure(figsize=(11,6))
plt.scatter(ment, y_jitter, s = 0.1)

In [ ]:
phys = []
for i in range(len(X["PhysHlth"])):
    phys.append(X["PhysHlth"][i] + np.random.uniform(-0.2,0.2))
plt.figure(figsize=(11,6))    
plt.scatter(phys, y_jitter, s = 0.1)
plt.xlabel("Number of Good Physical Health Days (Last 30)")
plt.ylabel("Diabetes")
plt.title("Physical Health vs. Diabetes Diagnosis")
plt.savefig("physhlth")

In [ ]:
sns.countplot(x="Age", hue="Diabetes", data=df)
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Diabetes by Age', fontsize = 14)
#plt.savefig('age')

In [ ]:
sns.countplot(x="Education", hue="Diabetes", data=df)
plt.xlabel('Education Level')
plt.ylabel('Count')
plt.title('Diabetes by Education Level', fontsize = 14)
#plt.savefig("edu")

In [ ]:
sns.countplot(x="Income", hue="Diabetes", data=df)
plt.xlabel('Income (thousands of $)')
plt.ylabel('Count')
plt.title('Diabetes by Income', fontsize = 14)
plt.savefig('income')

## Model Fitting

In [ ]:
# Make train/validation/test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, test_size=0.2)

# Ravel y's to avoid DataConversionWarning
y_train = np.array(y_train)
y_train = y_train.ravel()
y_test = np.array(y_test)
y_test = y_test.ravel()

In [ ]:
# Fit Logistic Regression Model
log = LogisticRegression(max_iter=1000)
log.fit(X_train, y_train)
train_preds = log.predict(X_train)
test_preds = log.predict(X_test)
y_pred_proba = log.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)

# View metrics
print("AUC: ", auc)
print("Train Accuracy: ", accuracy_score(y_train, train_preds))
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("F1 Score: ", f1_score(y_test, test_preds))

In [ ]:
# Fit Random Forest Model
rf = RandomForestClassifier(min_samples_split=30)
rf.fit(X_train, y_train)
test_preds = rf.predict(X_test)
train_preds = rf.predict(X_train)
y_pred_proba = rf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC: ", auc)
print("Train Accuracy: ", accuracy_score(y_train, train_preds))
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("F1 Score: ", f1_score(y_test, test_preds))

In [ ]:
# Fit Gradient Boosting Model
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
test_preds = gb.predict(X_test)
train_preds = gb.predict(X_train)
y_pred_proba = gb.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC: ", auc)
print("Train Accuracy: ", accuracy_score(y_train, train_preds))
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("F1 Score: ", f1_score(y_test, test_preds))

In [ ]:
# Fit Ada Boost Model
ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
test_preds = ada.predict(X_test)
train_preds = ada.predict(X_train)
y_pred_proba = ada.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC: ", auc)
print("Train Accuracy: ", accuracy_score(y_train, train_preds))
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("F1 Score: ", f1_score(y_test, test_preds))

In [ ]:
# Fit Extra Trees Model
extra = ExtraTreesClassifier(min_samples_split=20)
extra.fit(X_train, y_train)
test_preds = extra.predict(X_test)
train_preds = extra.predict(X_train)
y_pred_proba = extra.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC: ", auc)
print("Train Accuracy: ", accuracy_score(y_train, train_preds))
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("F1 Score: ", f1_score(y_test, test_preds))

In [ ]:
# Fit KNN Classifier Model
# NOTE: This model ran for hours and never finished for me
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
test_preds = knn.predict(X_test)
train_preds = knn.predict(X_train)
print("Train Accuracy: ", accuracy_score(y_train, train_preds))
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("F1 Score: ", f1_score(y_test, test_preds))

In [ ]:
# Fit Support Vector Classifier Model
# NOTE: This model ran for hours and never finished for me
svc = SVC()
svc.fit(X_train, y_train)
test_preds = svc.predict(X_test)
train_preds = svc.predict(X_train)
print("AUC: ", auc)
print("Train Accuracy: ", accuracy_score(y_train, train_preds))
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("F1 Score: ", f1_score(y_test, test_preds))

In [ ]:
# Fit Gauusian Naive Bayes Model
gaus = GaussianNB()
gaus.fit(X_train, y_train)
test_preds = gaus.predict(X_test)
train_preds = gaus.predict(X_train)
y_pred_proba = gaus.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC: ", auc)
print("Train Accuracy: ", accuracy_score(y_train, train_preds))
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("F1 Score: ", f1_score(y_test, test_preds))

### Hyperparameter Tuning

In [ ]:
# Create Pipe and parameter grid
pipe = pipe = Pipeline([
    ('grad', GradientBoostingClassifier())
])
params = {
  'grad__loss':('log_loss','exponential'), 
  'grad__learning_rate':(0.05, 0.1, 0.2),
  'grad__n_estimators':(50, 100, 150, 200),
  'grad__criterion':('friedman_mse','squared_error')
}

# Run the grid search and output the best hyperparameters
gs = GridSearchCV(pipe, param_grid=params, scoring='accuracy', cv=10)
gs.fit(X_train, y_train)
gs.best_params_

In [ ]:
# Fit new model with tuned hyperparameters
gb = GradientBoostingClassifier(criterion="friedman_mse",
                                learning_rate=0.2,
                                loss='exponential',
                                n_estimators=200)
gb.fit(X_train, y_train)
preds = gb.predict(X_test)
y_pred_proba = gb.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC: ", auc)
accuracy_score(y_test, preds)

## ANN Exploration

In [ ]:
# Set seed
tf.random.set_seed(42)

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=keras.optimizers.Adam(
                    hp.Choice('learning_rate',
                              values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
# Tuning the hyperparameters via a random search
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    directory="ANN_hyperparameter_tuning")

tuner.search(X_train, y_train,
             epochs = 5,
             validation_split=0.2)



In [ ]:
# Outputs the best hyperparameters
print("Best # of neurons in dense layer: ", tuner.get_best_hyperparameters(num_trials=1)[0].get('units'))
print("Best Learning Rate: ", tuner.get_best_hyperparameters(num_trials=1)[0].get('learning_rate'))

## Final Model

In [ ]:
# Fit logistic regression model
log = LogisticRegression(max_iter=1000)
log.fit(X_train, y_train)
train_preds = log.predict(X_train)
test_preds = log.predict(X_test)
y_pred_proba = log.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)

# Get all metrics from logistic regression model
print("AUC: ", auc)
print("Train Accuracy: ", accuracy_score(y_train, train_preds))
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("F1 Score: ", f1_score(y_test, test_preds))
print(recall_score(y_test, test_preds))
print(precision_score(y_test, test_preds))


In [ ]:
# Find and plot SHAP values for each factor
explainer = shap.TreeExplainer(gb)
shap_values = explainer.shap_values(X_train)
exp = shap.Explanation(
    values=shap_values,
    base_values=explainer.expected_value,
    data=X_train,
    feature_names=X.columns
)
shap.plots.beeswarm(exp, show=False, color_bar=False)
plt.colorbar()
plt.title("SHAP Values from GradientBoosing Model")
plt.ylabel("Factors")
plt.tight_layout()
plt.savefig("shap")
plt.show()


In [ ]:
# Print out 5 each of true and false positives as well as true and false negatives
y_test_pred = gb.predict(X_test)
print("Some True Positives: " + str(np.where((y_test_pred == 1) & (y_test == 1))[0][:5])) # true positive
print("Some True Positives: " + str(np.where((y_test_pred == 0) & (y_test == 0))[0][:5])) # true negative
print("Some False Positives: " + str(np.where((y_test_pred == 1) & (y_test == 0))[0][:5])) # false positive
print("Some False Negatives: " + str(np.where((y_test_pred == 0) & (y_test == 1))[0][:5])) # false negative

In [ ]:
# Create dictionary of each factor and it's coefficient estimate
my_dictionary = {}
coefficients = pd.Series(log.coef_[0])
for key, value in zip(X.columns, coefficients):
    my_dictionary[key] = value

# Print results
my_dictionary